In [3]:
using Distances, Statistics
using MultivariateStats
using PyPlot
using WordTokenizers
using TextAnalysis
using DelimitedFiles
using Embeddings
using DataFrames
using CSV

In [4]:
crashes = CSV.read("/Users/asharma19/Downloads/CrashTable.csv")

,Top Crashes,Crash Count,Error
,Int64,Int64,String⍰
1,20917683101,23465,Exception has been thrown by the target of an invocation.
2,18490782808,16577,Container Save Exception
3,21446158798,5290,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.
4,21302206595,3613,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.
5,20253196742,2818,Could not initialize class com.intuit.cg.fs.fps.api.Region
6,20227402431,2288,External component has thrown an exception.
7,19245349646,2055,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.
8,20570994344,1910,Native components of SQL Server Compact engine are not loaded. Please reinstall Microsoft SQL Server Compact.
9,20203474017,1842,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.


In [5]:
crashes.Error = recode(crashes.Error,"undefined"=>missing)

7052-element Array{Union{Missing, String},1}:
 "Exception has been thrown by the target of an invocation."                                                                          
 "Container Save Exception"                                                                                                           
 "Attempted to read or write protected memory. This is often an indication that other memory is corrupt."                             
 "Attempted to read or write protected memory. This is often an indication that other memory is corrupt."                             
 "Could not initialize class com.intuit.cg.fs.fps.api.Region"                                                                         
 "External component has thrown an exception."                                                                                        
 "Attempted to read or write protected memory. This is often an indication that other memory is corrupt."                             
 "Native 

In [6]:
crashes = dropmissing(crashes, :Error)

,Top Crashes,Crash Count,Error
,Int64,Int64,String
1,20917683101,23465,Exception has been thrown by the target of an invocation.
2,18490782808,16577,Container Save Exception
3,21446158798,5290,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.
4,21302206595,3613,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.
5,20253196742,2818,Could not initialize class com.intuit.cg.fs.fps.api.Region
6,20227402431,2288,External component has thrown an exception.
7,19245349646,2055,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.
8,20570994344,1910,Native components of SQL Server Compact engine are not loaded. Please reinstall Microsoft SQL Server Compact.
9,20203474017,1842,Attempted to read or write protected memory. This is often an indication that other memory is corrupt.


In [7]:
crashes[!, :Error]

6764-element Array{String,1}:
 "Exception has been thrown by the target of an invocation."                                                                          
 "Container Save Exception"                                                                                                           
 "Attempted to read or write protected memory. This is often an indication that other memory is corrupt."                             
 "Attempted to read or write protected memory. This is often an indication that other memory is corrupt."                             
 "Could not initialize class com.intuit.cg.fs.fps.api.Region"                                                                         
 "External component has thrown an exception."                                                                                        
 "Attempted to read or write protected memory. This is often an indication that other memory is corrupt."                             
 "Native components of SQ

In [8]:
crashes[!, :Error] = replace.(crashes[!, :Error], r"\\|\n|\r|_|,|—|\." => " ")
crashes[!, :Error] = replace.(crashes[!, :Error], r"[\"*();!`\'0-9-]" => "")

6764-element Array{String,1}:
 "Exception has been thrown by the target of an invocation "                                                              
 "Container Save Exception"                                                                                               
 "Attempted to read or write protected memory  This is often an indication that other memory is corrupt "                 
 "Attempted to read or write protected memory  This is often an indication that other memory is corrupt "                 
 "Could not initialize class com intuit cg fs fps api Region"                                                             
 "External component has thrown an exception "                                                                            
 "Attempted to read or write protected memory  This is often an indication that other memory is corrupt "                 
 "Native components of SQL Server Compact engine are not loaded  Please reinstall Microsoft SQL Server Compac

In [12]:
const embtable = load_embeddings(GloVe{:en},1) # or load_embeddings(FastText_Text) or ...
#Function to return the index of the word in the embedding (returns 0 if the word is not found)
const get_word_index = Dict(word=>ii for (ii,word) in enumerate(embtable.vocab))
embeddings = embtable.embeddings
vocab = embtable.vocab
vec_size, vocab_size = size(embeddings)
println("Loaded embeddings, each word is represented by a vector with $vec_size features. The vocab size is $vocab_size")
vec_idx(s) = findfirst(x -> x == s, vocab)
function vec(s)
    if vec_idx(s)!= nothing
        embeddings[:, vec_idx(s)]
    end
end
# To compare the similarity of document vectors
cosine(x,y) = 1 - cosine_dist(x, y)
function closest(v, n=20)
    list = [(x,cosine(embeddings'[x,:], v)) for x in 1:size(embeddings)[2]]
    topn_idx = sort(list, by = x -> x[2], rev = true)[1:n]
    return [vocab[a] for (a,_) in topn_idx]
end
# Function to get a mean sentence vector
function sentvec(s)
    local arr = []
    for w in split(sentences[s])
        if vec(w) != nothing
            push!(arr, vec(w))
        end
    end
    if length(arr) == 0
        ones(Float32, (50,1))*999
    else
        mean(arr)
    end
end
function closest_sent(input_str, n=100)
    mean_vec_input = mean([vec(w) for w in split(input_str)])
    list = [(x,cosine(mean_vec_input, sentvec(x))) for x in 1:length(sentences)]
    topn_idx=sort(list, by = x -> x[2], rev = true)[1:n]
    return [sentences[a] for (a,_) in topn_idx]
end
function match_docs(; kws...)
    args = Args(; kws...)
    txt = open(args.path) do file
        read(file, String)
    end
    println("Loaded $(args.path), length=$(length(txt)) characters")
    txt = replace(txt, r"\n|\r|_|,|—" => " ")
    txt = replace(txt, r"[\"""*();!]" => "")
    sd = StringDocument(txt)
    prepare!(sd, strip_whitespace)
    global sentences = split_sentences(sd.text)
    i = 1
    for s in 1:length(sentences)
        if length(split(sentences[s]))>3
            sentences[i]=lowercase(replace(sentences[s], "."=>""))
            i += 1
        end
    end
    save_vector(sentences)
    return closest_sent(args.string)
end
function save_vector(sentences)
    i = length(sentences) + 1
    sent_vecs = []
    for s in 1:length(sentences)
        i == 1 ? sent_vecs=sentvec(s) : push!(sent_vecs,sentvec(s))
    end
    writedlm( "data/sent_vec.csv",  sent_vecs, ',')
    writedlm( "data/sentences.csv",  sentences, ',')
end
function closest_sent_pretrained(pretrained_arr, input_str, n=20)
    mean_vec_input = mean([vec(w) for w in split(input_str)])
    list = [(x,cosine(mean_vec_input, pretrained_arr[x,:])) for x in 1:length(sentences)]
    topn_idx = sort(list, by = x -> x[2], rev=true)[1:n]
    return [sentences[a] for (a,_) in topn_idx]
end
function load_and_match(sentences_path, sent_vecs_path, string)
    sentences = readdlm("sentences.csv", '!', String, header=false)
    sent_vecs = readdlm("sent_vec.csv", ',', Float32, header=false)
    return closest_sent_pretrained(sent_vecs, string)
end


Loaded embeddings, each word is represented by a vector with 50 features. The vocab size is 400000


LoadError: syntax: cannot juxtapose string literal

In [13]:
sentences = crashes[!, :Error]
sentvec(10)

50-element Array{Float32,1}:
  0.39382502 
 -0.027129117
  0.43173927 
 -0.15439376 
  0.25947985 
  0.20323718 
 -0.32047498 
 -0.0861025  
  0.17393562 
 -0.03249225 
 -0.012721248
  0.18091875 
  0.050838   
  ⋮          
  0.45049262 
 -0.25194627 
  0.038054377
  0.028798006
  0.4671725  
  0.10219687 
 -0.2408925  
  0.011267881
 -0.05143312 
 -0.20767753 
  0.23728438 
 -0.30543002 

In [50]:
str_match = "at IntuitTaxInterop.PrintInterop.GetPrintList(PrintSelections selection, List`1 formsetIds)
   at Intuit.Ctg.Wte.Service.Printer.ViewModel.PrintTree.ToggleTree.UpdateTree()"
str_match = replace(str_match, r"\n|\r|_|,|—|\." => " ")
str_match = replace(str_match, r"[\"*();!`]" => "")

"at IntuitTaxInterop PrintInterop GetPrintListPrintSelections selection  List1 formsetIds    at Intuit Ctg Wte Service Printer ViewModel PrintTree ToggleTree UpdateTree"

In [14]:
closest_sent("memory leak")

100-element Array{String,1}:
 "The request failed due to a fatal device hardware error   "                                                                                                               
 "disk I/O error"                                                                                                                                                           
 "Could not find file C: data cloud Dropbox Personal personal   us   matt windows Documents TurboTax TurboTax TempT    pdf "                                                
 "Could not find file C: Users awais OneDrive Documents TurboTax TurboTax TempT    pdf "                                                                                    
 "Could not find file C: Users awais OneDrive Documents TurboTax TurboTax TempT    pdf "                                                                                    
 "Could not find file C: Users awais OneDrive Documents TurboTax TurboTax TempT    pdf "                  